# Flight distances

We have latitude or longitude values for all 81 provinces and we have to calculate distances between them.

## Get Coordinates

Copy this method below to load _coordinates.csv_ file into the flights _DataFrame_ first.

In [ ]:
coordinates <- read.csv("coordinates.csv", row.names = 1, fileEncoding = "UTF-8")
head(coordinates)

## Reminder

A data frame is actually a _list_ of equal-length _vectors_.

Make sure your data is in dataframe format.

In [ ]:
class(coordinates)
coordinates <- as.data.frame(coordinates)

In order to reach the latitudes of the coordinates dataframe:

In [ ]:
coordinates$lat
coordinates[[1]]
coordinates[,1]

In order to reach the latitude of a given city in dataframe:

In [ ]:
coordinates["Adana", "lat"]
coordinates[1, 1]

In order to reach the coordinates of a given city in dataframe:

In [ ]:
coordinates[c("Adana", "Adıyaman"),]
coordinates[c("Adana", "Adıyaman"), 1:2]

In order to reach the cities whose latitude is between 36 and 37.

In [ ]:
coordinates[(coordinates$lat > 36 & coordinates$lat < 37),]

## Calculate Distance

In order to calculate distance between two points on earth we use the function below:

\begin{align}
a &= sin^2(\Delta\varphi/2) + cos\varphi_1 * cos\varphi_2 * sin^2(\Delta\lambda/2) \\
c &= 2 * atan2(\sqrt{a}, \sqrt{1-a}) \\
d &= R * c
\end{align}

where $\varphi$ is latitude and $\lambda$ is longitude, and $R$ represents radius which is  6.371km.

We will write a function named *calculate\_distance* which will calculate the distance between two points on earth using the formula above.

 - $\Delta$ means the difference (between latitude or longitude values)
 - $d$ is the distance we try to find
 
**Note**: Function above works on radians but given coordinates are in degrees. In order to convert them, you should use $r = d * \pi / 180$ where $d$ is degree and $r$ is radians.

**Hint**:

```R
# Function in R
atan2()
```
#### Example usage
```R
> calculate_distance(coordinates["Adana",], coordinates["Adıyaman",])
274.130329948729
```

In [ ]:
# http://www.movable-type.co.uk/scripts/latlong.html
calculate_distance <- function(prov1, prov2)
{
  rad_earth <- 6371 # radius of earth
  
  coor1 <- prov1 * pi / 180
  coor2 <- prov2 * pi / 180
  
  lat1 <- coor1$lat
  lng1 <- coor1$lng
  
  lat2 <- coor2$lat
  lng2 <- coor2$lng
  
  dlat <- lat1 - lat2
  dlng <- lng1 - lng2
  
  a <- sin(dlat / 2)^2 + cos(lat1) * cos(lat2) * sin(dlng / 2)^2
  
  cc <- 2 * atan2(sqrt(a), sqrt(1-a))
  
  distance <- rad_earth * cc
  
  return(distance)
}

## Generate Distance Matrix

We will write the function *distance_mat_gen* which will calculate all the distances of a given coordinates and return them as a data frame. In our case, column and row names should be corresponding provinces.

#### Example Usage

In [ ]:
distance_mat <- distance_mat_gen(coordinates)
head(distance_mat)

In [ ]:
 nprov <- nrow(coordinates)
nprov
  distance_mat = as.data.frame(matrix(nrow = nprov, ncol = nprov),
                               row.names=row.names(coordinates))

In [ ]:
distance_mat_gen <- function(coordinates)
{
  nprov <- nrow(coordinates)
  # Generate an empty dataframe with proper row names
  distance_mat = as.data.frame(matrix(nrow = nprov, ncol = nprov),
                               row.names=row.names(coordinates))
  # Set column names of generated empty data frame
  colnames(distance_mat) <- row.names(coordinates)
  # Iterate over provinces
  for(row in 1:nprov) {
    # Since symmetric matrix, calculating one side is enough
    for(col in row:nprov) {
      actual_dist <- calculate_distance(coordinates[row,], coordinates[col,])
      # since distances are symmetric...
      distance_mat[row, col] <- actual_dist
      distance_mat[col, row] <- actual_dist
    }
  }
  
  return(distance_mat)
}